In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./info.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Kit_Part_Number'] != ''].drop_duplicates(['Kit_JOIN_TCPK'],
                                                                 keep='first',
                                                                 ignore_index=True)

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, output_null
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'func': 'getbuyersguide',
                        'payload': '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[a, 'Kit_Type_Code'] + '","partkey":"' + input_.loc[a, 'Kit_Part_Key'] + '"}}}',
                        'api_json_request': '1'}
                
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_data['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))
                    
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for i in range(len(list_tr)):
                    dict_vehicle[str(i)] = {}
                    list_td = [str(_).strip() for _ in list_tr[i].xpath('./td/text()')]
                    for j in range(len(list_td)):
                        dict_vehicle[str(i)][str(j)] = list_td[j]
                
                # = = = = = = = = = = = = = = =
                
                vehicle_json = json.dumps(dict_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for serial in dict_vehicle:
                    vehicle = ''
                    for serial_2 in dict_vehicle[serial]:
                        vehicle += ' ' + dict_vehicle[serial][serial_2]
                    list_vehicle.append(vehicle.strip())

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                                         'Kit_Vehcile_Json': vehicle_json,
                                         'Kit_Vehcile': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Kit_JOIN_TCPK'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Kit_JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./kit_vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Kit_Type_Code', 'Kit_Part_Key'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./kit_vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：926

[ok] - 5724;8306420
[尝试次数：1] - [剩余数量：914] - [当前时间：10:07:58]

[ok] - 5736;10110036
[尝试次数：3] - [剩余数量：913] - [当前时间：10:07:59]

[ok] - 5724;7093400
[尝试次数：2] - [剩余数量：912] - [当前时间：10:07:59]

[ok] - 5723;8314716
[尝试次数：9] - [剩余数量：911] - [当前时间：10:08:01]

[ok] - 5728;8307288
[尝试次数：4] - [剩余数量：910] - [当前时间：10:08:01]

[ok] - 5736;10110024
[尝试次数：12] - [剩余数量：909] - [当前时间：10:08:02]

[ok] - 5728;10110040
[尝试次数：11] - [剩余数量：908] - [当前时间：10:08:03]

[ok] - 5736;8307908
[尝试次数：1] - [剩余数量：907] - [当前时间：10:08:03]

[ok] - 5728;10110020
[尝试次数：19] - [剩余数量：906] - [当前时间：10:08:04]

[ok] - 5728;10195092
[尝试次数：2] - [剩余数量：905] - [当前时间：10:08:05]

[ok] - 10465;10195124
[尝试次数：3] - [剩余数量：904] - [当前时间：10:08:05]

[ok] - 5728;10110016
[尝试次数：21] - [剩余数量：903] - [当前时间：10:08:05]

[ok] - 5724;10110012
[尝试次数：22] - [剩余数量：902] - [当前时间：10:08:05]

[ok] - 5736;8307904
[尝试次数：14] - [剩余数量：901] - [当前时间：10:08:06]

[ok] - 5728;10519392
[尝试次数：1] - [剩余数量：900] - [当前时间：10:08:07]

[ok] - 5736;10110032
[尝试次数：26] - [剩余数量：899] - [当前时间：10:08:07

[ok] - 5736;8307936
[尝试次数：2] - [剩余数量：780] - [当前时间：10:09:05]

[ok] - 5568;8527968
[尝试次数：2] - [剩余数量：779] - [当前时间：10:09:05]

[ok] - 5736;8308264
[尝试次数：8] - [剩余数量：778] - [当前时间：10:09:06]

[ok] - 5722;8313608
[尝试次数：4] - [剩余数量：777] - [当前时间：10:09:07]

[ok] - 5728;10890336
[尝试次数：17] - [剩余数量：776] - [当前时间：10:09:07]

[ok] - 5728;8307896
[尝试次数：18] - [剩余数量：775] - [当前时间：10:09:08]

[ok] - 11859;8921276
[尝试次数：11] - [剩余数量：774] - [当前时间：10:09:09]

[ok] - 5568;11158033
[尝试次数：7] - [剩余数量：773] - [当前时间：10:09:10]

[ok] - 11860;11158181
[尝试次数：1] - [剩余数量：772] - [当前时间：10:09:10]

[ok] - 5728;11158185
[尝试次数：4] - [剩余数量：771] - [当前时间：10:09:11]

[ok] - 5728;8307560
[尝试次数：5] - [剩余数量：770] - [当前时间：10:09:11]

[ok] - 5736;11158189
[尝试次数：2] - [剩余数量：769] - [当前时间：10:09:11]

[ok] - 5723;11158217
[尝试次数：2] - [剩余数量：768] - [当前时间：10:09:14]

[ok] - 5723;11158213
[尝试次数：7] - [剩余数量：767] - [当前时间：10:09:14]

[ok] - 5736;11158193
[尝试次数：9] - [剩余数量：766] - [当前时间：10:09:15]

[ok] - 11859;11492513
[尝试次数：3] - [剩余数量：765] - [当前时间：10:09:16]

[ok] - 57

[ok] - 5728;14047829
[尝试次数：4] - [剩余数量：646] - [当前时间：10:10:10]

[ok] - 5723;8314944
[尝试次数：6] - [剩余数量：645] - [当前时间：10:10:10]

[ok] - 5564;8310396
[尝试次数：18] - [剩余数量：644] - [当前时间：10:10:10]

[ok] - 5724;13060761
[尝试次数：4] - [剩余数量：643] - [当前时间：10:10:10]

[ok] - 5724;14047605
[尝试次数：4] - [剩余数量：642] - [当前时间：10:10:11]

[ok] - 5728;14047833
[尝试次数：4] - [剩余数量：641] - [当前时间：10:10:11]

[ok] - 5721;15583773
[尝试次数：13] - [剩余数量：640] - [当前时间：10:10:11]

[ok] - 5736;8308280
[尝试次数：17] - [剩余数量：639] - [当前时间：10:10:12]

[ok] - 5724;14047609
[尝试次数：2] - [剩余数量：638] - [当前时间：10:10:12]

[ok] - 5723;14047941
[尝试次数：4] - [剩余数量：637] - [当前时间：10:10:13]

[ok] - 5728;14047809
[尝试次数：3] - [剩余数量：636] - [当前时间：10:10:13]

[ok] - 5728;13570105
[尝试次数：1] - [剩余数量：635] - [当前时间：10:10:14]

[ok] - 5728;14047849
[尝试次数：3] - [剩余数量：634] - [当前时间：10:10:14]

[ok] - 5736;14047837
[尝试次数：8] - [剩余数量：633] - [当前时间：10:10:14]

[ok] - 5736;8308212
[尝试次数：23] - [剩余数量：632] - [当前时间：10:10:15]

[ok] - 5722;14047957
[尝试次数：1] - [剩余数量：631] - [当前时间：10:10:15]

[ok] - 5

[ok] - 5728;8307508
[尝试次数：11] - [剩余数量：513] - [当前时间：10:11:10]

[ok] - 12910;15383933
[尝试次数：2] - [剩余数量：512] - [当前时间：10:11:11]

[ok] - 12910;15583717
[尝试次数：3] - [剩余数量：511] - [当前时间：10:11:11]

[ok] - 5564;8921408
[尝试次数：17] - [剩余数量：510] - [当前时间：10:11:12]

[ok] - 11860;8308236
[尝试次数：12] - [剩余数量：509] - [当前时间：10:11:12]

[ok] - 5564;10981408
[尝试次数：20] - [剩余数量：508] - [当前时间：10:11:12]

[ok] - 18586;8317284
[尝试次数：8] - [剩余数量：507] - [当前时间：10:11:13]

[ok] - 18586;8317280
[尝试次数：11] - [剩余数量：506] - [当前时间：10:11:13]

[ok] - 5723;8313604
[尝试次数：7] - [剩余数量：505] - [当前时间：10:11:13]

[ok] - 5736;15383317
[尝试次数：7] - [剩余数量：504] - [当前时间：10:11:15]

[ok] - 10709;14047817
[尝试次数：4] - [剩余数量：503] - [当前时间：10:11:16]

[ok] - 18586;15583713
[尝试次数：1] - [剩余数量：502] - [当前时间：10:11:16]

[ok] - 5564;8310268
[尝试次数：4] - [剩余数量：501] - [当前时间：10:11:16]

[ok] - 5728;15383281
[尝试次数：1] - [剩余数量：500] - [当前时间：10:11:17]

[ok] - 5728;15383277
[尝试次数：14] - [剩余数量：499] - [当前时间：10:11:18]

[ok] - 10466;13570097
[尝试次数：10] - [剩余数量：498] - [当前时间：10:11:19]



[ok] - 5723;10454636
[尝试次数：16] - [剩余数量：379] - [当前时间：10:12:29]

[ok] - 5723;9854584
[尝试次数：9] - [剩余数量：378] - [当前时间：10:12:31]

[ok] - 5724;8304580
[尝试次数：4] - [剩余数量：377] - [当前时间：10:12:31]

[ok] - 5723;8313584
[尝试次数：8] - [剩余数量：376] - [当前时间：10:12:31]

[ok] - 5728;8308168
[尝试次数：5] - [剩余数量：375] - [当前时间：10:12:31]

[ok] - 5728;8307464
[尝试次数：5] - [剩余数量：374] - [当前时间：10:12:32]

[ok] - 11860;8308180
[尝试次数：2] - [剩余数量：373] - [当前时间：10:12:32]

[ok] - 5723;8313676
[尝试次数：27] - [剩余数量：372] - [当前时间：10:12:33]

[ok] - 11859;8921272
[尝试次数：3] - [剩余数量：371] - [当前时间：10:12:33]

[ok] - 5723;8313836
[尝试次数：3] - [剩余数量：370] - [当前时间：10:12:33]

[ok] - 5589;8313588
[尝试次数：2] - [剩余数量：369] - [当前时间：10:12:34]

[ok] - 5723;8313736
[尝试次数：2] - [剩余数量：368] - [当前时间：10:12:35]

[ok] - 5728;3950475
[尝试次数：12] - [剩余数量：367] - [当前时间：10:12:35]

[ok] - 5723;8314648
[尝试次数：7] - [剩余数量：366] - [当前时间：10:12:35]

[ok] - 5724;8304644
[尝试次数：5] - [剩余数量：365] - [当前时间：10:12:35]

[ok] - 5723;8314608
[尝试次数：1] - [剩余数量：364] - [当前时间：10:12:36]

[ok] - 5722;831406

[ok] - 5728;8307260
[尝试次数：2] - [剩余数量：245] - [当前时间：10:14:17]

[ok] - 5332;14047901
[尝试次数：1] - [剩余数量：244] - [当前时间：10:14:18]

[ok] - 5736;8307944
[尝试次数：12] - [剩余数量：243] - [当前时间：10:14:19]

[ok] - 5723;8314748
[尝试次数：2] - [剩余数量：242] - [当前时间：10:14:22]

[ok] - 5722;8314744
[尝试次数：4] - [剩余数量：241] - [当前时间：10:14:23]

[ok] - 5724;8306004
[尝试次数：1] - [剩余数量：240] - [当前时间：10:14:23]

[ok] - 5724;8304688
[尝试次数：1] - [剩余数量：239] - [当前时间：10:14:24]

[ok] - 5728;8307296
[尝试次数：2] - [剩余数量：238] - [当前时间：10:14:24]

[ok] - 5722;8314664
[尝试次数：18] - [剩余数量：237] - [当前时间：10:14:26]

[ok] - 5728;8307300
[尝试次数：7] - [剩余数量：236] - [当前时间：10:14:27]

[ok] - 5724;8305916
[尝试次数：2] - [剩余数量：235] - [当前时间：10:14:28]

[ok] - 5722;8314536
[尝试次数：5] - [剩余数量：234] - [当前时间：10:14:28]

[ok] - 5723;8314424
[尝试次数：6] - [剩余数量：233] - [当前时间：10:14:28]

[ok] - 5728;8307920
[尝试次数：8] - [剩余数量：232] - [当前时间：10:14:29]

[ok] - 5736;8307968
[尝试次数：6] - [剩余数量：231] - [当前时间：10:14:31]

[ok] - 5722;8314700
[尝试次数：6] - [剩余数量：230] - [当前时间：10:14:31]

[ok] - 5724;8305996
[

[ok] - 5736;8308100
[尝试次数：1] - [剩余数量：110] - [当前时间：10:15:40]

[ok] - 5728;8307120
[尝试次数：1] - [剩余数量：109] - [当前时间：10:15:40]

[ok] - 5728;8307456
[尝试次数：1] - [剩余数量：108] - [当前时间：10:15:41]

[ok] - 5728;8307452
[尝试次数：1] - [剩余数量：107] - [当前时间：10:15:41]

[ok] - 5723;8314200
[尝试次数：4] - [剩余数量：106] - [当前时间：10:15:41]

[ok] - 10466;8307476
[尝试次数：1] - [剩余数量：105] - [当前时间：10:15:42]

[ok] - 5728;8308160
[尝试次数：1] - [剩余数量：104] - [当前时间：10:15:43]

[ok] - 5188;8306444
[尝试次数：1] - [剩余数量：103] - [当前时间：10:15:43]

[ok] - 10709;8308196
[尝试次数：1] - [剩余数量：102] - [当前时间：10:15:43]

[ok] - 5728;8308200
[尝试次数：1] - [剩余数量：101] - [当前时间：10:15:44]

[ok] - 5728;8307480
[尝试次数：1] - [剩余数量：100] - [当前时间：10:15:44]

[ok] - 5193;8314932
[尝试次数：1] - [剩余数量：99] - [当前时间：10:15:45]

[ok] - 5589;8314828
[尝试次数：1] - [剩余数量：98] - [当前时间：10:15:45]

[ok] - 51028;9659032
[尝试次数：1] - [剩余数量：97] - [当前时间：10:15:45]

[ok] - 5723;8314904
[尝试次数：1] - [剩余数量：96] - [当前时间：10:15:46]

[ok] - 5724;9659048
[尝试次数：1] - [剩余数量：95] - [当前时间：10:15:46]

[ok] - 5728;9659068
[尝试次数：